In [110]:
import pandas as pd
import glob
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import manipulating_data as md

In [3]:
path = glob.glob('**/*dengue_features_train.csv', recursive=True) [0]

In [4]:
deng_train = pd.read_csv(path)

In [5]:
deng_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [6]:
class DataSet() :
    def __init__(self, path) :
        self.path = path
        self.data = pd.read_csv(path)

In [7]:
data = DataSet(path)
data.data.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [118]:
class CategoricalVariable() :


    def __init__(self, column : pd.Series) :
        self.column = column
    
    @staticmethod
    def ordinal_encode(column: pd.Series, get_values_mapping = False) -> pd.Series:

        encoder = OrdinalEncoder()
        encoded_data = encoder.fit_transform(pd.DataFrame(column)).flatten()
        
        return pd.Series(encoded_data, index=column.index, name=column.name)

    @staticmethod
    def one_hot_encode(column: pd.Series, show_mapping = False) -> pd.DataFrame:

        encoder = OneHotEncoder(sparse=False)
        fitted_encoder = encoder.fit(pd.DataFrame(column))
        transformed = fitted_encoder.transform(pd.DataFrame(column))
        inverse_transformation = encoder.inverse_transform(transformed)

        if show_mapping :

            values_to_representations = { i.tolist()[0] : t.tolist() for t, i in\
                                        zip(transformed, inverse_transformation) }
            
            return values_to_representations
        
        
        return pd.DataFrame(transformed, columns=fitted_encoder.get_feature_names())

    
    def encode_data(self, method) -> pd.DataFrame:
        if method == 'ordinal':
            encoded_df = CategoricalVariable.ordinal_encode(self.column)
        elif method == 'one_hot':
            encoded_df = CategoricalVariable.one_hot_encode(self.column)
        else:
            raise ValueError(f"Encoding method {method} not recognized.")
        return encoded_df  # Return the correct variable here




In [119]:
categorical_col = CategoricalVariable(deng_train['city'])

In [120]:
encoded_data = categorical_col.encode_data('one_hot')

c:\Users\Predator\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [121]:
encoded_data

,x0_iq,x0_sj
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
1451,1.0,0.0
1452,1.0,0.0
1453,1.0,0.0
1454,1.0,0.0


In [104]:
mapping

{'sj': [0.0, 1.0], 'iq': [1.0, 0.0]}

In [112]:
help(md.manipulating_data_structures)

Help on module manipulating_data.manipulating_data_structures in manipulating_data:

NAME
    manipulating_data.manipulating_data_structures

FUNCTIONS
    are_all_scalars(input_list)
        "
        Checks if all the items in a list are scalars (int, float, str).
        
        Parameter
        ---------
        input_list : list.
        
        Returns
        -------
        bool.
            True if all the items in a list are scalars, False otherwise.
        
        Example
        -------
        are_all_scalars([ 1, 'a', 3.77 ])
            
            >>> True
        
        are_all_scalars([ 1, 'a', [ 3.77 ] ])
            
            >>> False
    
    check_if_frames_are_equal(df_a, df_b) -> bool
        Checks if two dataframes are equal (have the same columns),
        regardless of the order of the columns.
        
        Parameters
        ----------
        df_a : pandas.DataFrame.
            First dataframe.
        df_b : pandas.DataFrame.
            

In [113]:
rows = []

for c in deng_train['city']:
    rows.append(mapping[c])

md.check_if_frames_are_equal(pd.DataFrame(rows, columns=encoded_data.columns), encoded_data)

True

In [54]:
encoder = OneHotEncoder(sparse=False)
fitted_encoder = encoder.fit(pd.DataFrame(['green', 'blue']))
transformed = fitted_encoder.transform(pd.DataFrame(['green', 'blue']))
transformed

array([[0., 1.],
       [1., 0.]])

In [62]:
encoder.inverse_transform(transformed)

array([['green'],
       ['blue']], dtype=object)

{'green': [0.0, 1.0], 'blue': [1.0, 0.0]}